In [2]:
import json 
import os 
import cv2
from pprint import pprint
import requests
import numpy as np
import random
import math
import albumentations as A
from tqdm import tqdm
import uuid
import requests
from annotations import LabelMe
import base64
from matplotlib import pyplot as plt
%matplotlib inline

/home/hbdesk/labelstudio_convert/venv/lib/python3.12/site-packages/albumentations/check_version.py:51: UserWarning: Error fetching version info <urlopen error [Errno 101] Network is unreachable>
  data = fetch_version_info()


In [3]:
LABEL_STUDIO_KEY="eaffb8f4719efb17f9b227ed56fee991c4d06a0a"
LABEL_STUDIO_URL="http://192.168.231.52:8080/"
OCR_SERVER="http://localhost:3000/en"


def string_from_img(image_path: str):
    with open(image_path, "rb") as image_file:
        files = { "file": (image_path, image_file, "image/jpeg") }
        response = requests.post(OCR_SERVER, files=files)
        if response.status_code==200:
            res_string = response.json()["string"]
            return res_string
        else:
            return None

def bgr2_3grey(image: np.ndarray):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_bgr = cv2.merge([gray_image, gray_image, gray_image])
    return gray_bgr

def convertImg2b64(imgpath):
    with open(imgpath, "rb") as img_file:
        imgstring = base64.b64encode(img_file.read())
    return imgstring

def xywh2xyxy(xywh, original_width, original_height):
    """
    Convert [x, y, w, h] in percentage format to xyxy absolute coordinates.

    Parameters:
    xywh (list): List of [x, y, w, h] as percentages, where x and y represent the top-left corner.
    original_width (int): Original width of the image.
    original_height (int): Original height of the image.

    Returns:
    tuple: (x_min, y_min, x_max, y_max) in absolute coordinates.
    """
    x_percent, y_percent, width_percent, height_percent = xywh

    x_min = (x_percent / 100) * original_width
    y_min = (y_percent / 100) * original_height
    x_max = x_min + (width_percent / 100) * original_width
    y_max = y_min + (height_percent / 100) * original_height

    return x_min, y_min, x_max, y_max

def get_img_from_studio(image_path, save_path="./weapon_img"):
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    download_url = f"{LABEL_STUDIO_URL}{image_path}"
    headers = {
        'Authorization': f'Token {LABEL_STUDIO_KEY}'
    }
    response = requests.get(download_url, headers=headers)
    if response.status_code == 200:
        filename = os.path.basename(image_path)
        save_path = os.path.join(save_path, filename)
        with open(save_path, 'wb') as f:
            f.write(response.content)
            return save_path
        print(f"Image successfully downloaded as {filename}")
    else:
        print(f"Failed to download image. Status code: {response.status_code}")
        return None

In [4]:
with open("./bbox_tall2.json", "r") as loadjson:
    data = json.loads(loadjson.read())
pprint(data[0])

{'annotations': [{'completed_by': 1,
                  'created_at': '2024-11-07T01:32:20.845676Z',
                  'draft_created_at': '2024-11-07T01:31:34.162268Z',
                  'ground_truth': False,
                  'id': 8993,
                  'import_id': None,
                  'last_action': None,
                  'last_created_by': None,
                  'lead_time': 262.752,
                  'parent_annotation': None,
                  'parent_prediction': None,
                  'prediction': {},
                  'project': 12,
                  'result': [{'from_name': 'label',
                              'id': 'CGdZlN2Ql-',
                              'image_rotation': 0,
                              'origin': 'manual',
                              'original_height': 21,
                              'original_width': 92,
                              'to_name': 'image',
                              'type': 'rectanglelabels',
                           

In [5]:
# get all annotations with string pairs
all_label_pairs = []
for anno in data:
    for _a in anno["annotations"][0]["result"]:
        if _a["type"] == "textarea":
            all_label_pairs.append(anno)
pprint(all_label_pairs[0])

{'annotations': [{'completed_by': 1,
                  'created_at': '2024-11-07T10:02:09.227810Z',
                  'draft_created_at': '2024-11-07T10:01:03.922716Z',
                  'ground_truth': False,
                  'id': 10210,
                  'import_id': None,
                  'last_action': None,
                  'last_created_by': None,
                  'lead_time': 289.98199999999997,
                  'parent_annotation': None,
                  'parent_prediction': None,
                  'prediction': {},
                  'project': 12,
                  'result': [{'from_name': 'label',
                              'id': '_d0oE7rshw',
                              'image_rotation': 0,
                              'origin': 'manual',
                              'original_height': 20,
                              'original_width': 133,
                              'to_name': 'image',
                              'type': 'rectanglelabels',
              

In [6]:
char_bbox = []
for _annotation in tqdm(all_label_pairs):
    all_bboxes = []
    label_string = None
    db_fname = _annotation["data"]["image"]
    image_name = get_img_from_studio(db_fname, "/home/hbdesk/labelstudio_convert/labelme_bbox_string/")
    original_image = cv2.imread(image_name)
    for _anno in _annotation["annotations"][0]["result"]:
        final_pairs = []

        if len(_annotation["annotations"][0]["result"]) > 0:
            if _anno["type"] == "textarea":
                label_string = _anno["value"]["text"][0]
            if _anno["type"] == "rectanglelabels":
                height = _anno["original_height"]
                width = _anno["original_width"]
                xywh = [_anno["value"]["x"], _anno["value"]["y"], _anno["value"]["width"], _anno["value"]["height"]]
                x1 , y1 , x2, y2  = xywh2xyxy(xywh, width, height)
                all_bboxes.append([x1,y1,x2,y2])
    _annotation = LabelMe(width, height, os.path.basename(image_name))
    base64_img = convertImg2b64(image_name)
    _annotation.label["imageData"] = base64_img.decode("utf-8")
    # print("LABEL: ", label_string)
    if label_string is not None:
        for idx, box in enumerate(all_bboxes):
            try:
                _label_string = label_string.replace(" ", "")
                # print("curchar : ", _label_string)
                curchar = _label_string[idx]
                x1,y1,x2,y2 = [int(x) for x in box]
                crop = original_image[y1:y2, x1:x2]
                h, w ,c = crop.shape
                pad_width = max(0, 32 - w)
                pad_height = max(0, 32 - h)
                top = pad_height // 2
                bottom = pad_height - top
                left = pad_width // 2
                right = pad_width - left
                _pad = np.pad(crop, ((top, bottom), (left, right), (0, 0)), mode='constant', constant_values=0)
                # _resize = cv2.cvtColor(cv2.resize(crop, (112, 112)), cv2.COLOR_BGR2GRAY)
                _pad = cv2.cvtColor(_pad, cv2.COLOR_BGR2GRAY)
                _annotation.add_label(curchar, [[x1,y1], [x2,y2]], "rectangle")
                # if not os.path.exists(f"bbox_chars/{curchar}"):
                #     os.makedirs(f"bbox_chars/{curchar}")
                # cv2.imwrite(f"bbox_chars/{curchar}/{uuid.uuid4().hex}.png", _pad)
            except Exception as _:
                continue
    else: 
        continue
    json_file = os.path.splitext(image_name)[0] + ".json"
    with open(os.path.join("/home/hbdesk/labelstudio_convert/labelme_bbox_string/", json_file), "w") as writejson:
        writejson.write(json.dumps(_annotation.label, indent=2))
    # pprint(_annotation.label)

              

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 1045/1045 [01:01<00:00, 17.00it/s]
